## LCA PREPROCESSING

optional extension, for automatic pretty-formatting of code:

In [ ]:
%load_ext nb_black

filter out warnings if wanted:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Import modules

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys

from math import isnan

# plotting modules
import matplotlib.pyplot as plt

# self-written modules:
sys.path.append("./lca/")
import utils
import LCA_file_reading # note that this script is not on GitHub yet, due to patient-specific information in script
import preprocessing
import ontology_based_harmonizing
import scib_excerpts
import dim_reduction

#### version info

Print python path:

In [ ]:
sys.executable

Print python version:

In [ ]:
sys.version

print package versions (can do this manually too instead of using utils, if wanted)

In [ ]:
utils.print_loaded_module_versions(globals().values())

Print module versions:

## import data:

NOTE that the script LCA_file_reading is not made available on GitHub yet, since some of the data in there are patient-specific and cannot be published yet.

#### 1) adatas with one AnnData per sample

i.e. Reyfman/Misharin, VieiraBraga/Nawijn_nasal, VieiraBraga/Teichmann, Morse/Lafyatis, Misharin_new

In [ ]:
data_dir = "/path/to/dir/lung_cell_atlas/"

In [ ]:
# # read in each of the datasets:
adatas_per_sample_dict = dict()
# Reyfman et al.
project_name_Reyfman = "Northwestern_Misharin_2018Reyfman"
project_dir_Reyfman = data_dir + project_name_Reyfman + "/"
print("\n" + project_name_Reyfman)
adatas_per_sample_dict["Misharin"] = LCA_file_reading.read_file_Reyfman(
    project_dir=project_dir_Reyfman
)
# VieiraBraga/Nawijn nasal:
project_name_VieiraBraga = "Sanger_Teichmann_2019VieiraBraga"
project_dir_VieiraBraga = data_dir + project_name_VieiraBraga + "/"
print("\n" + project_name_VieiraBraga, "UMCG nasal")
adatas_per_sample_dict[
    "Nawijn_nasal"
] = LCA_file_reading.read_file_VieiraBraga_UMCG_nasal(
    project_dir_VieiraBraga, donor_type="healthy", verbose=True
)
# VieiraBraga/Sanger samples:
print("\n" + project_name_VieiraBraga, "Sanger parenchyma")
adatas_per_sample_dict["Teichmann"] = LCA_file_reading.read_file_VieiraBraga_Sanger(
    project_dir_VieiraBraga
)
# # Raredon et al
# project_name_Raredon = "Yale_Niklason_2019Raredon"
# project_dir_Raredon = data_dir + project_name_Raredon + "/"
# print("\n" + project_name_Raredon)
# adatas_per_sample_dict["Raredon"] = LCA_file_reading.read_file_Raredon(
#     project_dir=project_dir_Raredon
# )
# Morse et al
project_name_Morse = "Pittsburgh_Lafyatis_2019Morse"
project_dir_Morse = data_dir + project_name_Morse + "/"
print("\n" + project_name_Morse)
adatas_per_sample_dict["Lafyatis"] = LCA_file_reading.read_file_Morse(
    project_dir=project_dir_Morse
)
# Misharin new:
project_name_Misharin_new = "Misharin_new"
project_dir_Misharin_new = data_dir + project_name_Misharin_new + "/"
adatas_per_sample_dict["Misharin_new"] = LCA_file_reading.read_file_Misharin_new(
    project_dir_Misharin_new
)

In [ ]:
# adatas_per_sample_dict

In [ ]:
# list number of samples per dict:
for dataset_name, adatas in adatas_per_sample_dict.items():
    print(dataset_name.upper() + ":")
    print(len(adatas.keys()), "samples")
    total_cells = 0
    for sample, adata in adatas.items():
        n_cells = adata.shape[0]
        total_cells = total_cells + n_cells
    print(str(total_cells) + " cells\n")
    #     # check if each dataset has the required columns.
    #     # take only first sample:
    #     print(adatas[list(adatas.keys())[0]].obs.columns)
#     print(adatas[list(adatas.keys())[0]].var.columns)

#### 2) adatas with one AnnData per dataset:

i.e. Vieira-Braga/Nawijn_lung, Madissoon/Meyer, Habermann/Kropski, Travaglini/Krasnow, Deprez/Barbry, Goldfarbmuren/Seibold

In [ ]:
adatas_per_dataset = dict()

In [ ]:
# Vieira-Braga/Nawijn lung:
project_name_VieiraBraga = "Sanger_Teichmann_2019VieiraBraga"
project_dir_VieiraBraga = data_dir + project_name_VieiraBraga + "/"
print(project_name_VieiraBraga, "UMCG lung")
adatas_per_dataset["Nawijn_lung"] = LCA_file_reading.read_file_VieiraBraga_UMCG_lung(
    project_dir=project_dir_VieiraBraga
)

In [ ]:
# Madissoon et al
project_name_Madissoon = "Sanger_Meyer_2019Madissoon"
project_dir_Madissoon = data_dir + project_name_Madissoon + "/"
print(project_name_Madissoon)
adatas_per_dataset["Meyer"] = LCA_file_reading.read_file_Madissoon(
    project_dir=project_dir_Madissoon
)

In [ ]:
# Habermann et al
project_name_Habermann = "Vanderbilt_Kropski_bioRxivHabermann"
project_dir_Habermann = data_dir + project_name_Habermann + "/"
print(project_name_Habermann)
adatas_per_dataset["Kropski"] = LCA_file_reading.read_file_Habermann(
    project_dir=project_dir_Habermann
)

In [ ]:
# Travaglini et al
project_name_Travaglini = "Stanford_Krasnow_bioRxivTravaglini"
project_dir_Travaglini = data_dir + project_name_Travaglini + "/"
print(project_name_Travaglini)
adatas_per_dataset["Krasnow"] = LCA_file_reading.read_file_Travaglini(
    project_dir=project_dir_Travaglini
)

In [ ]:
# Deprez et al
project_name_Deprez = "CNRS_Barbry_bioRxivDeprez"
project_dir_Deprez = data_dir + project_name_Deprez + "/"
print(project_name_Deprez)
adatas_per_dataset["Barbry"] = LCA_file_reading.read_file_Deprez(
    project_dir=project_dir_Deprez
)

In [ ]:
# Goldfarbmuren et al
project_name_Goldfarbmuren = "NJH_Seibold_2020Goldfarbmuren"
project_dir_Goldfarbmuren = data_dir + project_name_Goldfarbmuren + "/"
print(project_name_Goldfarbmuren)
adatas_per_dataset["Seibold"] = LCA_file_reading.read_file_Goldfarbmuren()

In [ ]:
for dataset_name, adata_object in adatas_per_dataset.items():
    print(dataset_name, adata_object.obs.columns, adata_object.var.columns)

### add up duplicate genes and remove genes with 0 counts:

first the per-sample AnnData dictionary:

In [ ]:
for dataset_name, adatas in adatas_per_sample_dict.items():
    print(dataset_name.toupper() + ":\n")
    for sample_name, adata in adatas.items():
        print(sample_name + ":")
        # filter out genes without counts
        n_genes_before = adata.shape[1]
        sc.pp.filter_genes(adata, min_counts=1)
        # drop annotation label that is automatically created:
        adata.var.drop("n_counts", axis=1, inplace=True)
        n_genes_after = adata.shape[1]
        print("number of genes removed:", n_genes_before - n_genes_after)
        adatas[sample_name] = preprocessing.add_up_duplicate_gene_name_columns(adata)
        print(adata.shape, "\n")

now the per-dataset AnnData dictionary:

In [ ]:
for dataset_name, adata in adatas_per_dataset.items():
    print(dataset_name + ":")
    n_genes_before = adata.shape[1]
    sc.pp.filter_genes(adata, min_counts=1)
    # drop annoation column that is automatically created:
    adata.var.drop("n_counts", axis=1, inplace=True)
    n_genes_after = adata.shape[1]
    print("number of genes removed:", n_genes_before - n_genes_after)
    adatas_per_dataset[dataset_name] = preprocessing.add_up_duplicate_gene_name_columns(
        adata
    )
    print(str(adata.shape) + "\n")

### pool per-sample AnnData objects to per-dataset AnnData objects:

In [ ]:
for dataset_name, adatas in adatas_per_sample_dict.items():
    print("Dataset: " + dataset_name)
    adata = sc.AnnData.concatenate(
        *adatas.values(),
        join="outer",
        batch_key="sample_temp",
        batch_categories=list(adatas.keys()),
        index_unique="_"
    )
    # remove sample name column:
    adata.obs.drop("sample_temp", axis=1, inplace=True)
    # set nan to zero
    adata.X = np.nan_to_num(adata.X)
    print(adata.shape)
    # shuffle rows:
    index_list = np.arange(adata.shape[0])
    np.random.shuffle(index_list)
    adata = adata[index_list]
    adatas_per_dataset[dataset_name] = adata.copy()

Check if datasets have GRCh37 or GRCh38 gene names:

In [ ]:
genes_37 = ["AC000032.2", "AC000036.4", "ZNF724P", "ZNF812", "AC000068.10"]
genes_38 = ["AC000032.1", "AC000036.1", "ZNF724", "ZNF812P", "AC000068.1"]
for gene_37, gene_38 in zip(genes_37, genes_38):
    print(gene_37)
    for dataset in adatas_per_dataset.keys():
        found_gene = ""
        if gene_37 in adatas_per_dataset[dataset].var.index:
            found_gene = found_gene + "37"
        if gene_38 in adatas_per_dataset[dataset].var.index:
            found_gene = found_gene + "38"
        print(dataset, found_gene)
    print("\n")

Now pool between datasets:

In [ ]:
adata = sc.AnnData.concatenate(
    *adatas_per_dataset.values(),
    join="outer",
    batch_key="dataset_temp",
    batch_categories=list(adatas_per_dataset.keys()),
    index_unique="_"
)
print(adata.shape)

remove columns that were automatically added when concatenating AnnDatas:

In [ ]:
adata.obs.drop(["dataset_temp"], axis=1, inplace=True)

In [ ]:
adata.shape

double check if all datasets had ensembl84 genes:

In [ ]:
genes_84 = ["AC000032.2", "AC000036.4", "ZNF724P", "AC000068.10"]
genes_93 = ["AC000032.1", "AC000036.1", "ZNF724", "AC000068.1"]
for gene_84, gene_93 in zip(genes_84, genes_93):
    print(gene_84)
    found_gene = ""
    if gene_84 in adata.var.index:
        found_gene = found_gene + "84"
    if gene_93 in adata.var.index:
        found_gene = found_gene + "93"
    print(found_gene)
    print("\n")

shuffle rows:

In [ ]:
adata.X = np.nan_to_num(adata.X)
index_list = np.arange(adata.shape[0])
np.random.shuffle(index_list)
adata = adata[index_list]

## correct abnormal gene names, and remove duplicate gene names
(also for compatibility with R)

In [ ]:
adata_backup = adata.copy()

In [ ]:
# correct gene names
renamer_dict = preprocessing.get_gene_renamer_dict(adata.var.index.tolist().copy())
n_genes_to_rename = np.sum(old != new for old, new in renamer_dict.items())
print(n_genes_to_rename, "GENES TO BE RENAMED (old, new):\n")
for old, new in renamer_dict.items():
    if len(new) > 0:
        print(old, new)
adata.var["original_gene_names"] = adata.var.index.tolist().copy()
translation_dict = dict(zip(adata.var.index, adata.var.index))
for gene_to_rename, new_name in renamer_dict.items():
    translation_dict[gene_to_rename] = new_name
adata.var.index = adata.var.index.map(translation_dict)

In [ ]:
# remove any new duplicates that might have emerged:
adata = preprocessing.add_up_duplicate_gene_name_columns(adata)

In [ ]:
del adata_backup

In [ ]:
adata

## Add cell annotations related to QC

Add cell annotations such as total counts per cell, percentage of mitochondrial reads etc. This has to be done before any filtering or normalizing is performed!

In [ ]:
# annotate with QC stuff:
adata = preprocessing.add_cell_annotations(adata)

## Add cell ontology annotation:

Original cell type labeling (as provided by dataset providers) will now be translated to the current version of the cell type ontology, consisting of 5 levels.

correct/check some dataset and cluster naming:

In [ ]:
# correct two incomplete cluster names Misharin dataset:
cells_to_change = [
    cell
    for cell, ann in zip(adata.obs.index, adata.obs.original_celltype_ann)
    if ann in ["C7_2_DC2_FCER1A", "C7_7_DC1_CLEC9A"]
]
if len(cells_to_change) > 0:
    print("adapting cell type annotations of wrongly named types from Sasha")
    # convert to list (not necessary for function)
    adata.obs.original_celltype_ann = list(adata.obs.original_celltype_ann.values)
    adata.obs.loc[cells_to_change, "original_celltype_ann"] = [
        sample + "_" + label
        for sample, label in zip(
            adata.obs.loc[cells_to_change, "sample"],
            adata.obs.loc[cells_to_change, "original_celltype_ann"],
        )
    ]

Make sure data nawijn is split to two, since they have different original annotations (see cell type ontology)



In [ ]:
adata.obs.dataset = list(adata.obs.dataset)
nawijn_lung_cells = adata.obs.apply(
    lambda x: x["lung_vs_nasal"] == "lung"
    and x["dataset"] == "UMCG_Nawijn_2019VieiraBraga",
    axis=1,
)
print("number of Nawijn lung cells with only general dataset annotation:", np.sum(nawijn_lung_cells))
nawijn_nasal_cells = adata.obs.apply(
    lambda x: x["lung_vs_nasal"] == "nasal"
    and x["dataset"] == "UMCG_Nawijn_2019VieiraBraga",
    axis=1,
)
print("number of Nawijn nasal cells with only general dataset annotation:", np.sum(nawijn_lung_cells))


adata.obs.loc[nawijn_lung_cells, "dataset"] = "UMCG_Nawijn_2019VieiraBraga_lung"
adata.obs.loc[nawijn_nasal_cells, "dataset"] = "UMCG_Nawijn_2019VieiraBraga_nasal"

First, import the .csv that contains the translations:  
NOTE that this file is not made publicly available yet, but will be made available with publication

In [ ]:
harmonizing_df = ontology_based_harmonizing.load_harmonizing_table(
    "/path/to/dir/LCA/ontologies/cell_type_ontologies/cell_type_ontology_20201012.csv"
)

Create a dataframe that contains each cell type name from the consensus ontology as indices, with their matching annotations at the other levels. This will simplify mapping:

In [ ]:
consensus_df = ontology_based_harmonizing.create_consensus_table(harmonizing_df)

create a dataframe that for each original celltype annotation (from all datasets pooled) provides the translation to the consensus ontology at all levels:

In [ ]:
celltype_translation_df = ontology_based_harmonizing.create_celltype_to_consensus_translation_df(
    adata, consensus_df, harmonizing_df, verbose=False
)

now translate the original annotations to the consensus in your AnnData:

In [ ]:
adata = ontology_based_harmonizing.consensus_annotate_anndata(
    adata, celltype_translation_df, verbose=True
)

### add sample/donor annotations from LCA metadata tables:

NOTE: this part of the script cannot be run yet, since you'll need a file with patient information that cannot be made public yet.

Note that the naming of the _samples_ is harmonized rather than the donor naming, so use sample names to copy metadata to AnnData object.

In [ ]:
metadata = preprocessing.get_sample_annotation_table_LCA()

note that [patient specific information, removed for GitHub version of script for now]

In [ ]:
samples_in_adata = sorted(set(adata.obs["sample"]))
samples_in_metadata = metadata.index
print("n samples in adata:", len(samples_in_adata))
if len(samples_in_metadata) != len(set(samples_in_metadata)):
    print("WARNING: DUPLICATE SAMPLE NAMES IN METADATA TABLE! THIS SHOULD BE FIXED.")
for sample in samples_in_adata:
    if sample not in samples_in_metadata:
        print(sample, "is in AnnData object but not in metadata. Check this.")
for sample in samples_in_metadata:
    if sample not in samples_in_adata:
        print(sample, "is in metadata but not in AnnData object. Check this.")

In [ ]:
metadata_columns_to_drop = ["IF AVAILABLE/ APPLICABLE -->"]

In [ ]:
metadata.drop(columns=metadata_columns_to_drop, inplace=True)

In [ ]:
for cat in metadata.columns:
    sample_to_cat_dict = dict(zip(metadata.index, metadata[cat]))
    adata.obs[cat] = adata.obs["sample"].map(sample_to_cat_dict)

check within-dataset diversity of technical covariates

In [ ]:
adata.obs.groupby("dataset").agg(
    {
        "cell ranger version ": "nunique",
        "disease status": "nunique",
        "fresh or frozen": "nunique",
        "known lung disease": "nunique",
        "sample type": "nunique",
        "sequencing platform": "nunique",
        "single cell platform": "nunique",
        "subject type": "nunique",
        "tissue dissociation protocol": "nunique",
    }
)

## splitting of datasets into separate batches, where necessary

Three datasets should be split into seperate batches:  
    - kropski/banovich: they come from two different institutes, can be derived from donor names  
    - lafyatis: includes both 10xv1 and 10xv2  
    - seibold: includes both 10xv2 and 10xv3

In [ ]:
sorted(set(adata.obs.dataset))

In [ ]:
sample_to_dataset_df = adata.obs.groupby("sample").agg(
    {"sample": "first", "dataset": "first", "single cell platform": "first"}
)
sample_to_dataset_dict = dict(
    zip(sample_to_dataset_df["sample"], sample_to_dataset_df.dataset)
)

kropski/banovich

In [ ]:
samples_dna = {
    sample: "Vanderbilt_Kropski_bioRxivHabermann_dna"
    for sample in ["F01157", "F01174", "F01365", "F01366", "F01367"]
}
samples_vand = {
    sample: "Vanderbilt_Kropski_bioRxivHabermann_vand"
    for sample in ["HD65", "HD66", "HD67", "HD68", "F00409", "HD70", "F01394"]
}
sample_to_dataset_dict.update(samples_dna)
sample_to_dataset_dict.update(samples_vand)

lafyatis

In [ ]:
samples_lafy_v1 = {
    sample: "Pittsburgh_Lafyatis_2019Morse_10Xv1"
    for sample in sample_to_dataset_df.index
    if sample_to_dataset_df.loc[sample, "dataset"] == "Pittsburgh_Lafyatis_2019Morse"
    and sample_to_dataset_df.loc[sample, "single cell platform"] == "10x_3'_v1"
}
samples_lafy_v2 = {
    sample: "Pittsburgh_Lafyatis_2019Morse_10Xv2"
    for sample in sample_to_dataset_df.index
    if sample_to_dataset_df.loc[sample, "dataset"] == "Pittsburgh_Lafyatis_2019Morse"
    and sample_to_dataset_df.loc[sample, "single cell platform"] == "10x_3'_v2"
}
sample_to_dataset_dict.update(samples_lafy_v1)
sample_to_dataset_dict.update(samples_lafy_v2)

seibold

In [ ]:
samples_seibold_v2 = {
    sample: "NJH_Seibold_2020Goldfarbmuren_10Xv2"
    for sample in sample_to_dataset_df.index
    if sample_to_dataset_df.loc[sample, "dataset"] == "NJH_Seibold_2020Goldfarbmuren"
    and sample_to_dataset_df.loc[sample, "single cell platform"] == "10x_3'_v2"
}
samples_seibold_v3 = {
    sample: "NJH_Seibold_2020Goldfarbmuren_10Xv3"
    for sample in sample_to_dataset_df.index
    if sample_to_dataset_df.loc[sample, "dataset"] == "NJH_Seibold_2020Goldfarbmuren"
    and sample_to_dataset_df.loc[sample, "single cell platform"] == "10x_3'_v3"
}
sample_to_dataset_dict.update(samples_seibold_v2)
sample_to_dataset_dict.update(samples_seibold_v3)

In [ ]:
adata.obs["dataset"] = adata.obs["sample"].map(sample_to_dataset_dict)

merge Nawijn dataset to one (for downstream analyses)

In [ ]:
datasets = sorted(set(adata.obs.dataset))
# convert Martijn's datasets to just one dataset:
dataset_to_dataset_mapping = dict(zip(datasets, datasets))
dataset_to_dataset_mapping[
    "UMCG_Nawijn_2019VieiraBraga_lung"
] = "UMCG_Nawijn_2019VieiraBraga"
dataset_to_dataset_mapping[
    "UMCG_Nawijn_2019VieiraBraga_nasal"
] = "UMCG_Nawijn_2019VieiraBraga"
adata.obs.dataset = adata.obs.dataset.map(dataset_to_dataset_mapping)

In [ ]:
sorted(set(adata.obs.dataset))

## remove diseased subject(s):

In [ ]:
# filter out asthma patients:
# T84: seibold childhood asthma
# THD0002 and VUHD65: asthma from Kropski/Banovich
asthma_subjects = ["T84", "THD0002", "VUHD65"]
n_cells_before = adata.shape[0]
filter = [subj not in asthma_subjects for subj in adata.obs.subject_ID]
if sum(filter) < adata.shape[0]:
    adata = adata[filter, :].copy()
n_cells_after = adata.shape[0]
print(
    "number of cells removed by filtering out asthmatic subjects:",
    n_cells_before - n_cells_after,
)

## Metadata cleanup:

To correct (part of this might have already been fixed with corrected reading scripts, check that next time I run this notebook):  
- missing lung_vs_nasal annotation for Seibold data (fixed in read_file script)
- barbry column is "enrichment", all others are "enrichment "  
- same for cellranger version: harmonize. Also, 3.2.0 should be 3.0.2 for Barbry
- barbry data includes 'H', 'F' and 'M' as sex. translate to male/female
- misharin data misses 3' info
- missing institute info for kropski/banovich data
- cell viability should be numerical: remove trailing "%" and change to numbers

In [ ]:
seibold_cells = adata[adata.obs["last_author/PI"] == "Seibold", :].obs.index
barbry_cells = adata[adata.obs["last_author/PI"] == "Barbry/Leroy", :].obs.index
misharin_cells = adata.obs.loc[
    [x in ["Misharin", "Misharin/Budinger"] for x in adata.obs["last_author/PI"]], :
].index.values

In [ ]:
# set seibold data to lung
adata.obs.loc[seibold_cells, "lung_vs_nasal"] = "lung"
# fix barbry column misnaming, and mistake in cell ranger version:
if "enrichment" in adata.obs.columns:
    print("adapting enrichment and column")
    adata.obs["enrichment "] = adata.obs["enrichment "].tolist()
    adata.obs.loc[barbry_cells, "enrichment "] = "No"
    adata.obs.drop(columns=["enrichment"], inplace=True)
if "cell ranger version" in adata.obs.columns:
    print("adapting cell ranger version and column")
    adata.obs.loc[barbry_cells, "cell ranger version "] = "3.0.2"
    adata.obs.drop(columns=["cell ranger version"], inplace=True)
if list(set(adata.obs.loc[barbry_cells, :]["cell ranger version "])) != ["3.0.2"]:
    print("adapting cell ranger version")
    adata.obs.loc[barbry_cells, "cell ranger version "] = "3.0.2"
# translate sex annotations from Barbry to common terminology
sex_dict = {"female":"female","male":"male", "F":"female","H":"male","M":"male"}
adata.obs.sex = adata.obs.sex.map(sex_dict)
# set Misharin data to 3' (missing annotation)
adata.obs.loc[misharin_cells, "3' or 5'"] = "3'"
# generate institute annotation
dataset_to_institute_df = adata.obs.groupby("dataset").agg(
    {"dataset": "first", "Institute": "first"}
)
dataset_to_institute_dict = dict(
    zip(dataset_to_institute_df.dataset, dataset_to_institute_df["Institute"])
)
dataset_to_institute_dict[
    "Vanderbilt_Kropski_bioRxivHabermann_dna"
] = "Donor Network of Arizona"
dataset_to_institute_dict["Vanderbilt_Kropski_bioRxivHabermann_vand"] = "Vanderbilt"
adata.obs["Institute"] = adata.obs["dataset"].map(dataset_to_institute_dict)
# convert viability percentage annotation to float instead of string
def conv_string_perc_to_float(string_percentage):
    stripped_string = string_percentage.rstrip(" ")
    stripped_string = stripped_string.rstrip("%")
    return float(stripped_string)


conv_string_perc_to_float = np.vectorize(conv_string_perc_to_float)
adata.obs["cell viability %"] = conv_string_perc_to_float(b
    adata.obs["cell viability %"].values
)

run metadata_cleaner function to further harmonize metadata annotations:

In [ ]:
adata.obs = preprocessing.metadata_cleaner(adata.obs)

### Harmonize anatomical region:

first add prefix to annotations from Barbry data, since their naming is inconsistent with other dataset's naming. Not adding prefix will result in mix-ups of translations.

In [ ]:
# prefix barbry detailed annotation with coarse:
adata.obs["anatomical region detailed"] = adata.obs[
    "anatomical region detailed"
].tolist()
adata_barbry = adata[adata.obs["last_author/PI"] == "Barbry/Leroy", :].copy()
barbry_region_detailed_prefixed = [
    x + "_" + y
    for x, y in zip(
        adata_barbry.obs["anatomical region coarse"],
        adata_barbry.obs["anatomical region detailed"],
    )
]
adata.obs.loc[
    adata_barbry.obs.index, "anatomical region detailed"
] = barbry_region_detailed_prefixed
del adata_barbry

now harmonize anatomical region:

read in harmonizing table:

NOTE: this part of the script cannot be run yet, since the matching files cannot be made public yet

In [ ]:
harmonizing_df = ontology_based_harmonizing.load_harmonizing_table(
    "/path/to/dir/LCA/ontologies/anatomical_region_ontologies/LCA_ontologies_anatomical_region_ontology_20201120.csv"
)

create translation table:

In [ ]:
consensus_df = ontology_based_harmonizing.create_consensus_table(
    harmonizing_df, max_level=3
)

translate both levels (coarse and fine) to their harmonized counterpart:

In [ ]:
for res in ["coarse", "fine"]:
    translation_df = (
        ontology_based_harmonizing.create_celltype_to_consensus_translation_df(
            adata,
            consensus_df,
            harmonizing_df,
            verbose=False,
            ontology_type="anatomical_region_" + res,
        )
    )
    adata = ontology_based_harmonizing.consensus_annotate_anndata(
        adata,
        translation_df,
        verbose=True,
        max_ann_level=3,
        ontology_type="anatomical_region_" + res,
    )

merge coarse and fine annotations, so that we keep the finest annotation available for every sample:

In [ ]:
adata = ontology_based_harmonizing.merge_coarse_and_fine_anatomical_ontology_anns(
    adata, remove_harm_coarse_and_fine_original=True
)

### harmonize nan/None/"nan" etc, clean metadata with function

In [ ]:
# set all different types of None/NaN to np.nan
none_entries = adata.obs.applymap(utils.check_if_nan)
adata.obs = adata.obs.mask(none_entries.values)

In [ ]:
# now clean up metadata again? (Is that necessary?)
adata.obs = preprocessing.metadata_cleaner(adata.obs)

### check that BMI is numerical:

In [ ]:
adata.obs.groupby(["dataset", "subject_ID"]).agg({"BMI": "first"})

In [ ]:
adata.obs["BMI"].dtype

if dtype is not float, check the set:

In [ ]:
# set(adata.obs["BMI"])

In [ ]:
adata.obs["BMI"] = adata.obs["BMI"].values.astype("float")

### add age annotation (merging of age in years and age range)

In [ ]:
# add age as merger of 'age, in years' and 'age, range'
adata.obs["age"] = [
    preprocessing.age_converter(age, age_range)
    for age, age_range in zip(adata.obs["age, in years"], adata.obs["age, range"])
]

## ARMS pseudonymization

THIS IS CUT FROM THE SCRIPT, SINCE THE PSEUDONYMIZATION CANNOT BE MADE PUBLIC

## Add "study" annotation:

Study is a simplified version of dataset, in which studies are not split based on e.g. sequencing platform version (see e.g. Lafyatis datasets)

In [ ]:
dataset_to_study_mapper = {
    "Sanger_Meyer_2019Madissoon": "Meyer_2019",
    "Vanderbilt_Kropski_bioRxivHabermann_vand": "Kropski_2020",  # _vand",
    "CNRS_Barbry_bioRxivDeprez": "Barbry_2020",
    "Pittsburgh_Lafyatis_2019Morse_10Xv2": "Lafyatis_2019",  # _10Xv2",
    "NJH_Seibold_2020Goldfarbmuren_10Xv3": "Seibold_2020",  # _10Xv3",
    "Stanford_Krasnow_bioRxivTravaglini": "Krasnow_2020",
    "Misharin_new": "Misharin_unpubl",
    "Sanger_Teichmann_2019VieiraBraga": "Teichmann_2019",
    "UMCG_Nawijn_2019VieiraBraga": "Nawijn_2019",
    "NJH_Seibold_2020Goldfarbmuren_10Xv2": "Seibold_2020",  # _10Xv2",
    "Northwestern_Misharin_2018Reyfman": "Misharin_2018",
    "Vanderbilt_Kropski_bioRxivHabermann_dna": "Kropski_2020",  # _dna",
    "Pittsburgh_Lafyatis_2019Morse_10Xv1": "Lafyatis_2019",  # _10Xv1",
}

In [ ]:
adata.obs['study'] = adata.obs.dataset.map(dataset_to_study_mapper)

In [ ]:
adata.shape

## Sanity checks:

check if number of unique annotations per dataset, per category are as expected (these should be mostly "1")

In [ ]:
adata.obs.groupby("dataset").agg(
    {
        "cell ranger version ": "nunique",
        "disease status": "nunique",
        "fresh or frozen": "nunique",
        "known lung disease": "nunique",
        "sample type": "nunique",
        "sequencing platform": "nunique",
        "single cell platform": "nunique",
        "subject type": "nunique",
        "tissue dissociation protocol": "nunique",
    }
)

check which datasets are included

In [ ]:
set(adata.obs.dataset)

In [ ]:
set(adata.obs.study)

In [ ]:
set(adata.obs["last_author/PI"])

### check if all variables have values for all cells:

we only expect that for some of them, but good to check

In [ ]:
for cat in adata.obs.columns:
    if isinstance(adata.obs[cat].values, np.ndarray):
        print(cat, np.nan in adata.obs[cat].values, "nan" in adata.obs[cat].values)
    else:
        print(cat, adata.obs[cat].values.isna().any(), "nan" in adata.obs[cat].values)

### Check if donor and sample names occur in only one dataset each:

In [ ]:
temp = adata.obs.groupby("sample").agg({"dataset": "nunique"})
# check if sample names only occur in one dataset:
for sample in temp.index:
    if temp.loc[sample, "dataset"] != 1:
        print(str(sample) + ": this sample name occurs in multiple datasets")
temp = adata.obs.groupby("subject_ID").agg({"dataset": "nunique"})
for donor in temp.index:
    if temp.loc[donor, "dataset"] != 1:
        print(
            str(donor)
            + ": this subject_ID name occurs in "
            + str(temp.loc[donor, "dataset"])
            + " datasets!"
        )

### check if all values have only zeros as decimals:

store remainders of division by 1, count for each row number of entries for which remainder is not 0 (they should all be zero if data are integers)

In [ ]:
test = np.sum(adata.X.toarray() % 1 == 0, axis=1)

select only those rows of adata that have non-integer values:

In [ ]:
nonint_adata = adata[test != adata.shape[1], :].copy()

check shape, it should have zero rows

In [ ]:
nonint_adata.shape

if it doesn't have zero rows, then check which datasets have non-integer values (in that case we received non-raw counts from them):

In [ ]:
set(nonint_adata.obs.dataset)

write/read:

In [ ]:
# adata.write(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_RAW_annotated.h5ad"
# )

In [ ]:
# adata = sc.read(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_RAW_annotated.h5ad"
# )

### filter out cells with low total counts and genes expressed in low number of cells:

filter out all cells with fewer than 500 cells:

In [ ]:
n_cells_pre = adata.shape[0]
sc.pp.filter_cells(adata, min_counts=500)
n_cells_post = adata.shape[0]
print("Number of cells removed: " + str(n_cells_pre - n_cells_post))
print("Number of cells pre-filtering: " + str(n_cells_pre))
print("Number of cells post filtering: " + str(n_cells_post))
adata.shape

remove genes that are expressed in fewer than 5 cells after cell filtering:

In [ ]:
n_genes_pre = adata.shape[1]
sc.pp.filter_genes(adata, min_cells=5)
n_genes_post = adata.shape[1]
print("Number of genes removed: " + str(n_genes_pre - n_genes_post))
print("Number of genes pre-filtering: " + str(n_genes_pre))
print("Number of genes post filtering: " + str(n_genes_post))

### normalize and log-transform with SCRAN normalization:

To normalize with scran, we need to cluster first, so we'll go through standard (non-sophisticated) normalization for a temporary adata, then calculate the size-factors on there, and then use those size factors to normalize the original anndata. All of this is done in the SCRAN_normalize function.

In [ ]:
plt.hist(adata.obs["log10_total_counts"], bins=50)
plt.show()

In [ ]:
# started at 10:30 ran till 12:41

In [ ]:
adata_norm = scib_excerpts.SCRAN_normalize(adata, log_transform=False) # started at 

In [ ]:
adata_norm

check if counts layer has integers

In [ ]:
np.sum(adata_norm.layers['counts'][:10,:], axis=1)

check if new adata.X has non-integers (it should):

In [ ]:
np.sum(adata_norm.X[:10,:], axis=1)

check for random row and column if the size factor and original count lead to the corrected count:

In [ ]:
i = 4
j = 9

In [ ]:
adata.X[i,j]

In [ ]:
adata.layers['counts'][i,j]

In [ ]:
adata.obs.size_factors[i]

In [ ]:
adata.X[4,9] * adata.obs.size_factors[i]

write/read

In [ ]:
# adata_norm.write(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_SCRAN_normalized.h5ad"
# )

In [ ]:
# adata = sc.read(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_SCRAN_normalized.h5ad"
# )

### HIGHLY VARIABLE GENE SELECTION:

select highly variable genes. We will first calculate highly variable genes per sample, then look for the genes that are highly variable in all samples and select those, all samples but one, all samples but two, etc. until we have 2000 overall highly variable genes. Code from the scib_excerpts module is taken from https://github.com/theislab/scib. See also https://www.biorxiv.org/content/10.1101/2020.05.22.111161v2.

In [ ]:
adata_nonlog = adata.copy()

In [ ]:
hvgs = scib_excerpts.hvg_batch(adata, batch_key="dataset")

In [ ]:
# check selected hvgs, see if they make sense:
means = np.mean(adata_nonlog.X, axis=0)
variances = np.var(adata_nonlog.X.toarray(), axis=0)
dispersions = variances / means
boolean_to_color = {
    True: "crimson",
    False: "steelblue",
}  # make a dictionary that translates the boolean to colors
hvg_colors = [
    boolean_to_color[x in hvgs] for x in adata_nonlog.var.index
]  # 'convert' the boolean

In [ ]:
plt.scatter(
    np.log1p(means).tolist()[0], np.log(dispersions).tolist()[0], s=2, c=hvg_colors
)
plt.xlabel("log1p(mean)")
plt.ylabel("log(dispersion)")
plt.title("DISPERSION VERSUS MEAN")
plt.show()

In [ ]:
adata.var["highly_variable"] = [x in hvgs for x in adata.var.index]

In [ ]:
del adata_nonlog

In [ ]:
adata

read/write

In [ ]:
# adata.write(
#     "/path/to/dir/LCA_h5ads/Barbry_Krasnow_Kropski_Lafyatis_Meyer_Misharin_MisharinNew_Nawijn_Teichmann_SCRAN_normalized_HVGann.h5ad"
# )

In [ ]:
# adata = sc.read(
#     "/path/to/dir/LCA_h5ads/Barbry_Krasnow_Kropski_Lafyatis_Meyer_Misharin_MisharinNew_Nawijn_Teichmann_SCRAN_normalized_HVGann.h5ad"
# )

### log-transformation

perform log-transformation:

In [ ]:
sc.pp.log1p(adata)

### PCA

Perform principal component analysis for dimensionality reduction, to reduce noise, and the limit computation time in downstream steps:

In [ ]:
adata = sc.tl.pca(adata, n_comps=200, copy=True, use_highly_variable=True)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=200, show=False)

Look at variance explained per principal component. At least all PCs that precede the 'elbow' in the plot should be included:

In [ ]:
# check variance explained plots:
dim_reduction.PCA_var_explained_plots(adata)

In [ ]:
# I'll take ... PCs
adata = sc.tl.pca(adata, n_comps=50, copy=True, use_highly_variable=True)

check which genes have highest loadings for the first 10 PCs. This can be a useful first glance at which genes contribute most the the structure of your data:

In [ ]:
sc.pl.pca_loadings(adata, components=range(1, 11))

#### UMAP

now run UMAP for visualization:

In [ ]:
sc.pp.neighbors(adata, n_neighbors=50)
sc.tl.umap(adata)

In [ ]:
adata

plot QC factors and annotations:

In [ ]:
testfig = sc.pl.umap(
    adata,
    color=adata.obs.columns.tolist(),
    sort_order=False,
    hspace=0.5,
    #     wspace=0.65,
    ncols=2,
    return_fig=True,
    #     save="/path/to/dir/test.png",
)

In [ ]:
testfig.savefig(
    "/path/to/dir/Barbry_Krasnow_Kropski_Lafyatis_Meyer_Misharin_MisharinNew_Nawijn_Teichmann_log1p_umaps.png",
    bbox_inches="tight",
    dpi=300,
)

In [ ]:
sc.pl.umap(
    adata, color=["dataset", "ann_level_4", "ann_level_5",], ncols=1, hspace=1,
)

In [ ]:
sc.pl.umap(
    adata, color=["sample",], ncols=1, wspace=0.40,
)

read/write

In [ ]:
# # store result:
# adata.write(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_log1p.h5ad"
# )

In [ ]:
# adata = sc.read(
#     "/path/to/dir/LCA_h5ads/Barb_Kras_Krop_Lafy_Meye_Mish_MishNew_Nawi_Seib_Teic_log1p.h5ad"
# )

# END